In [1]:
import numpy as np
import pandas as pd
import time
from IPython.display import clear_output
#from numba import njit

In [2]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

using gpu 2


In [3]:
class Environment:
    def __init__(self, size, posreward = 2, negreward = -2):
        self.size = size
        self.environment = np.array([[['0'] * self.size[2]] * self.size[1]] * self.size[0])
        self.posreward = posreward
        self.negreward = negreward
        self.score = 0
        self.done = False
        self.action_space = self.environment.shape[0]*self.environment.shape[1]
        self.observation_space = self.environment.shape[0]*self.environment.shape[1]
        self.input_space = self.environment.shape[0],self.environment.shape[1]
        
    def resetField(self):
        self.environment = np.array([[['0'] * self.size[2]] * self.size[1]] * self.size[0])
        
    def resetEnvironment(self):
        self.resetField()
        self.score = 0
        self.done = False
    
    def GenerateEnvironment(self):
        return np.array([[['0'] * self.size[2]] * self.size[1]] * self.size[0])
    
    def genObs(self, env):
        
        
        return np.array((env == '0').sum(axis=2), dtype=np.float32)
    
    def step(self, env, action):
        #Save Old State
        newState = env.copy()
        actionPos = np.unravel_index(action, newState.shape)
        done = False
        
        #Make move
        if self.placeContainer(actionPos, newState):
            #If move is allowed reward
            newState[actionPos] = '#'
            reward = self.posreward
        else:
            #If move is not allowed punish
            reward = self.negreward
        
        #Add to total score
        self.score += reward
        
        #End game if field is all #s or if the user messed up.
        if np.all(newState == '#') or reward == self.negreward:
            done = True
            self.done = True
        return newState, reward, done
    
    def placeContainer(self, pos, env):
        if self.isLegal(pos, env) and not self.done:
            return True
        else:
            return False
        
    def isLegal(self, pos, env):
        IO = self.isOccupied(pos, env)
        IF = self.isFloating(pos, env)
        IIE = self.posIsInEnv(pos, env)
        NAS = self.hasNorthAndSouth(pos, env)
        #print(IO,IF,IIE,NAS)
        return not IO and not IF and IIE and not NAS
    
    def isOccupied(self, pos, env):
        if self.posIsInEnv(pos, env):
            return env[pos] == '#'
        else:
            return False
    
    def hasNorthAndSouth(self, pos, env):
        NC = self.isOccupied((pos[0],pos[1]-1,0), env)
        SC = self.isOccupied((pos[0],pos[1]+1,0), env)
        #print(NC,SC)
        return NC and SC
    
    def posIsInEnv(self, pos, env):
        x = 0 <= pos[0] < env.shape[0]
        y = 0 <= pos[1] < env.shape[1]
        z = 0 <= pos[2] < env.shape[2]  
        return x and y and z
    
    def isFloating(self, pos, env):
        return np.any(env[pos[0],pos[1],:pos[2]] == '0')

In [4]:
e = Environment(size=(1,3,1))
environment = e.GenerateEnvironment()
newenv, reward, done = e.step(environment,0)
#print(newenv)
newenv, reward, done = e.step(newenv,2)
print(newenv)
newenv, reward, done = e.step(newenv,1)
print(newenv)


[[['#']
  ['0']
  ['#']]]
[[['#']
  ['0']
  ['#']]]


# Model

Agent(
    -Environment
    -CNN
    -Trainer)
    
    
    
# Pseudo pseudocode
<code>N = 10</code>

<code>While True:</code><br>
<code>    //Gaming moment (play 10 games)</code><br>
<code>    for i in range(10):</code><br>
<code>        pred = agent.pred()</code><br>
<code>        agent.rememberMoves()</code><br>
<code>    //Learning moment (train on previous games if possible)</code><br>
<code>    agent.trainNN()</code><br>



# Convolutional Neural Network

In [8]:
class CNN(nn.Module):
    def __init__(self, lr, input_space, action_space):
        super().__init__()
        self.input_space = input_space
        self.action_space = action_space
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU())
        self.fc = nn.Linear(16*self.input_space, self.action_space)
        
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
        self.loss = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
        
    def forward(self, X):
        #Inladen in layer 1
        h = self.layer1(X)
        
        #Platslaan
        h = h.reshape(-1, 16*self.input_space)
        # Het plaatsje begon bij een 28 * 28 en is nu een 14* 14. Daarom is de waarde in reshape => 16*14*14
        y = self.fc(h)
        return y

In [22]:
class Agent():
    def __init__(self, gamma, lr, batch_size, max_mem_size=100000, size = (3,3,2),epsilon = 0.99, eps_end=0.01, eps_dec=5e-4):
        
        self.epsilon = epsilon
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        
        
        self.lr = lr
        self.gamma = gamma
        
        #Create Environment
        self.env = Environment(size)
        
        self.action_space = self.env.action_space
        self.observation_space = [self.env.observation_space]
        self.input_space = self.env.input_space
        #Create Convolutional Neural Network
        self.cnn = CNN(lr = lr, input_space= self.observation_space[0], action_space=self.action_space)
        
        #Create Trainer
        #self.trainer = Trainer(self.cnn, nn.CrossEntropyLoss(), metric=accuracy_score, gpu=True)
        
        #Memory variables
        self.mem_size = max_mem_size
        self.mem_cntr = 0
        self.batch_size = batch_size
        
        #Memory array variables
        self.state_memory = np.zeros((self.mem_size, 1, self.input_space[0], self.input_space[1]), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, 1, self.input_space[0], self.input_space[1]), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=bool)
        
    def chooseAction(self, observation):
        if np.random.random() > self.epsilon:
            #Turn observation into tensor
            state = T.tensor([[observation]]).to(self.cnn.device)
            #print(state)
            #Get action from neural network
            actions = self.cnn.forward(state)

            #Get maximum value and return index
            action = T.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)
        return action
        
    def rememberMoves(self, state, action, reward, state_, done):
        #Makes counter loop over if max has been reached.
        index = self.mem_cntr % self.mem_size
        
        #Save specific variables to arrays
        self.state_memory[index] = [state]
        self.new_state_memory[index] = [state_]
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = done
        
        #Increment memory counter
        self.mem_cntr += 1
        
    #TODO: We still need to add the trainer to this part.
    def trainNN(self):
        #If there's not enough memory for the batch size. Don't learn.
        if self.mem_cntr < self.batch_size:
            return
        
        #Check what the maximum size is of the memory
        max_mem = min(self.mem_cntr, self.mem_size)
        
        #Choose a random batch
        batch = np.random.choice(max_mem, self.batch_size, replace=False)
        
        #Get batch indices for the batch size. (aka an array of batch_size int32s)
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        
        #Load batch memory to device in tensors.
        state_batch = T.tensor(self.state_memory[batch]).to(self.cnn.device)
        new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.cnn.device)
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.cnn.device)
        terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.cnn.device)
        
    
        
        #Load action batch memory.
        action_batch = self.action_memory[batch]
        
        #Generate output for eval state.
        q_eval = self.cnn.forward(state_batch)[batch_index, action_batch]
        
        #Generate output for next state.
        q_next = self.cnn.forward(new_state_batch)
        
        #Done state gets reset for the terminal batch?? idk?
        q_next[terminal_batch] = 0.0
        
        
        #Apply rewards to next batch.
        q_target = reward_batch + self.gamma * T.max(q_next, dim=1)[0]
        
        loss = self.cnn.loss(q_target, q_eval).to(self.cnn.device)
        loss.backward()
        self.cnn.optimizer.step()
        
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min
    
    

In [26]:
a = Agent(0.3, 0.01, 32)
#env = a.env.GenerateEnvironment()


RewardHistory = []
#TOFIX: Check if memory is programmed right?
#TOFIX: Allow the training step to actually work.

#print(env)
done = False
for o in range(1000):
    for i in range(10):
        env = a.env.GenerateEnvironment()
        done = False
        while not done:
            action = a.chooseAction(a.env.genObs(env))
            newenv, reward, done = a.env.step(env,action)
            #print(reward)
            RewardHistory.append(reward)
            #print(newenv, reward, done)
            print(np.mean(RewardHistory[:-100]))
            a.rememberMoves(a.env.genObs(env), action, reward, a.env.genObs(newenv), done)
            env = newenv

    a.trainNN()
    
#for i in range(100):
    
    
#    print(a.chooseAction(np.zeros((3,3), dtype=np.float32)))

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0


-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0


-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0


-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0


-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0


-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0


KeyboardInterrupt: 

In [51]:
class Trainer:
    def __init__(self, model, lossfunction, metric=None, gpu=False):
        self.device = torch.device('cuda:0') if gpu else torch.device('cpu')
        self.model = model
        self.lossfunction = lossfunction
        self.metric = metric
        self.optimizer = AdamW(model.parameters())
    
    def setTrainValid(self,train_dl,valid_dl):
        self.train_dl = train_dl
        self.valid_dl = valid_dl
        
    def train_once(self):
        epochloss = 0
        epochn = 0
        torch.set_grad_enabled(True)
        for X, y in self.train_dl:
            X = X.to(self.device)
            y = y.to(self.device)
            self.optimizer.zero_grad()
            y_pred = self.model(X)
            l = self.lossfunction(y_pred, y)
            l.backward()
            self.optimizer.step()
            epochloss += l.item() * len(X)
            epochn += len(X)
            y_class = torch.argmax(y_pred, axis=1)
            metric_score = self.metric(y.cpu(), y_class.cpu())
        return epochloss / epochn,  metric_score
            
    
    def validate_once(self):
        epochloss = 0
        epochn = 0
        with torch.set_grad_enabled(False):
            for X, y in self.valid_dl:
                X = X.to(self.device)
                y = y.to(self.device)
                y_pred = model(X)
                l = self.lossfunction(y_pred, y)
                epochloss += l.item() * len(X)
                epochn += len(X)
                y_class = torch.argmax(y_pred, axis=1)
                metric_score = self.metric(y.cpu(), y_class.cpu())
        return epochloss / epochn, metric_score
    
    def train(self, epochs, lr=0.01):
        self.train_losses = []
        self.valid_losses = []
        for e in range(epochs):
            train_loss = self.train_once()
            valid_loss = self.validate_once()
            self.train_losses.append(train_loss)
            self.valid_losses.append(valid_loss)
            print(f'epoch {e} loss train {train_loss[0]} valid {valid_loss[0]} metric valid {valid_loss[1]}')
    
    def learning_curve(self):
        train_loss_list = []
        valid_loss_list = []
        for i in t.train_losses:
            train_loss_list.append(i[0])
        for v in t.valid_losses:
            valid_loss_list.append(v[0])
        plt.figure()
        plt.plot(range(len(train_loss_list)), train_loss_list, label='train')
        plt.plot(range(len(valid_loss_list)), valid_loss_list, label='valid')
        plt.legend()
        plt.ylabel('loss_cel')
        plt.xlabel('epochs')
    
    def validation_curve(self):
        train_metric_list = []
        valid_metric_list = []
        for i in t.train_losses:
            train_metric_list.append(i[1])
        for v in t.valid_losses:
            valid_metric_list.append(v[1])
        plt.figure()
        plt.plot(range(len(train_metric_list)), train_metric_list, label='train')
        plt.plot(range(len(valid_metric_list)), valid_metric_list, label='valid')
        plt.legend()
        plt.ylabel('accuracy_score')
        plt.xlabel('epochs')

In [ ]:
class Agent():
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size, n_actions, fc1_dims=128,fc2_dims=128, max_mem_size=100000, eps_end=0.01, eps_dec=5e-4):
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.lr = lr
        self.action_space = [i for i in range(n_actions)]
        self.mem_size = max_mem_size
        self.batch_size = batch_size
        self.mem_cntr = 0
        
        #self.Q_eval = DeepQNetwork(self.lr, n_actions=n_actions, input_dims=input_dims, fc1_dims = fc1_dims, fc2_dims=fc2_dims)
        self.CNN = CNN(action_space = action_space)
        #self.tr = Trainer()
        self.state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size,*input_dims), dtype=np.float32)
        
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)
        
    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = done
        
        self.mem_cntr += 1
    
    def choose_action(self, observation):
        if np.random.random() > self.epsilon:
            state = T.tensor([observation]).to(self.Q_eval.device)
            actions = self.CNN.forward(state)
            action = T.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)
        return action
    
    def learn(self):
        #If there's not enough memory for the batch size. Don't learn.
        if self.mem_cntr < self.batch_size:
            return
        
        #Check what the maximum size is of the memory
        max_mem = min(self.mem_cntr, self.mem_size)
        
        #Choose a random batch
        batch = np.random.choice(max_mem, self.batch_size, replace=False)
        
        #Get batch indices for the batch size. (aka an array of batch_size int32s)
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        
        #Load batch memory to device in tensors.
        state_batch = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)
        
        #Load action batch memory.
        action_batch = self.action_memory[batch]
        
        #Generate output for eval state.
        q_eval = self.CNN.forward(state_batch)[batch_index, action_batch]
        
        #Generate output for next state.
        q_next = self.CNN.forward(new_state_batch)
        
        #Done state gets reset for the terminal batch?? idk?
        q_next[terminal_batch] = 0.0
        
        #Convert to DataLoader
        
        
        #Apply rewards to next batch.
        q_target = reward_batch + self.gamma * T.max(q_next, dim=1)[0]
        
        loss = self.CNN.loss(q_target, q_eval).to(self.Q_eval.device)
        
        tr = Trainer(self.CNN, loss, train_dl, valid_dl, metric=None, gpu=True)
        
        tr.train(3, lr = self.lr)
        
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min
    